<a href="https://colab.research.google.com/github/Krishnaugale353/BERT-FIneTuning/blob/main/complaints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sherrytp_consumer_complaints_path = kagglehub.dataset_download('sherrytp/consumer-complaints')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/consumer-complaints/complaints.csv
/kaggle/input/consumer-complaints/complaints.json


In [ ]:
df=pd.read_csv("/kaggle/input/consumer-complaints/complaints.csv")

/tmp/ipykernel_30/2844941221.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/kaggle/input/consumer-complaints/complaints.csv")


In [ ]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"EQUIFAX, INC.",TX,78541.0,Servicemember,NaN,Web,2022-11-19,In progress,Yes,NaN,6222374
1,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the fair credit reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,30043.0,NaN,Consent provided,Web,2022-10-13,Closed with explanation,Yes,NaN,6079679
2,2022-10-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,This is not a duplicate nor is this complaint ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,90660.0,NaN,Consent provided,Web,2022-10-13,Closed with non-monetary relief,Yes,NaN,6076990
3,2022-10-12,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Personal information incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19148.0,NaN,Consent not provided,Web,2022-10-12,Closed with non-monetary relief,Yes,NaN,6080312
4,2022-11-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OH,44130.0,NaN,NaN,Web,2022-11-19,In progress,Yes,NaN,6222634


In [ ]:
df.shape

(3093136, 18)

In [ ]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [ ]:
df=df[["Product","Consumer complaint narrative"]]

In [ ]:
df.shape

(3093136, 2)

In [ ]:
df.isna().sum()

Product                               0
Consumer complaint narrative    1981806
dtype: int64

In [ ]:
df.isna().sum() / len(df)*100

Product                          0.000000
Consumer complaint narrative    64.071092
dtype: float64

In [ ]:
df=df.dropna()

In [ ]:
df.shape

(1111330, 2)

In [ ]:
df["Product"].nunique()

18

In [ ]:
df["Product"].unique()

array(['Credit reporting, credit repair services, or other personal consumer reports',
       'Debt collection',
       'Money transfer, virtual currency, or money service', 'Mortgage',
       'Credit card or prepaid card', 'Vehicle loan or lease',
       'Student loan', 'Checking or savings account',
       'Payday loan, title loan, or personal loan', 'Consumer Loan',
       'Bank account or service', 'Credit reporting', 'Credit card',
       'Prepaid card', 'Money transfers', 'Payday loan',
       'Other financial service', 'Virtual currency'], dtype=object)

In [ ]:
df["Product"].value_counts()/len(df)*100

Product
Credit reporting, credit repair services, or other personal consumer reports    46.215616
Debt collection                                                                 17.261300
Mortgage                                                                         8.790638
Credit card or prepaid card                                                      7.351372
Checking or savings account                                                      4.868671
Student loan                                                                     2.940711
Credit reporting                                                                 2.842360
Money transfer, virtual currency, or money service                               2.387500
Vehicle loan or lease                                                            1.786508
Credit card                                                                      1.695086
Bank account or service                                                          1.339386
Pa

In [ ]:
class_dict = {
    'Vehicle loan or lease': 'Loan',
    'Credit card or prepaid card': 'Card',
    'Credit reporting, credit repair services, or other personal consumer reports': 'Credit Report',
    'Money transfer, virtual currency, or money service': 'Services',
    'Mortgage': 'Others',
    'Payday loan, title loan, or personal loan': 'Loan',
    'Debt collection': 'Others',
    'Student loan': 'Loan',
    'Consumer Loan': 'Loan',
    'Credit card': 'Card',
    'Checking or savings account': 'Others',
    'Money transfers': 'Services',
    'Bank account or service': 'Services',
    'Credit reporting': 'Credit Report',
    'Prepaid card': 'Card',
    'Payday loan': 'Loan',
    'Other financial service': 'Services',
    'Virtual currency': 'Services'
}


In [ ]:
df.replace({"Product":class_dict},inplace=True)

In [ ]:
df

,Product,Consumer complaint narrative
1,Credit Report,In accordance with the fair credit reporting a...
2,Credit Report,This is not a duplicate nor is this complaint ...
81,Credit Report,It has come to my attention. That my personal ...
82,Credit Report,The Federal Trade Commission Bureau of Consume...
86,Credit Report,There were several inaccurate personal informa...
...,...,...
3093130,Card,"Automated calls from "" XXXX with Capital One '..."
3093131,Others,I have disputed my debts several times with no...
3093132,Others,My father died in XX/XX/XXXX. Left me his only...
3093133,Credit Report,cfbp i would Like to file a complaint on Exper...


In [ ]:
df["Product"].value_counts()/len(df)*100

Product
Credit Report    49.057976
Others           30.920609
Card              9.176932
Loan              6.955180
Services          3.889304
Name: count, dtype: float64

In [ ]:
import re

In [ ]:
def preprocessing_text(complaint):
    complaint = complaint.lower()

    # Remove special characters and digits
    complaint = re.sub(r'[^a-zA-Z\s]', '', complaint)
    complaint = re.sub(r'[x]{2,}', '', complaint)
    return complaint



In [ ]:
df["Consumer complaint narrative"]=df["Consumer complaint narrative"].apply(preprocessing_text)

In [ ]:
df["Consumer complaint narrative"].iloc[0]

'in accordance with the fair credit reporting act   account                                                                                                 has violated my rights usc section  a states that i have a right to my privacy   usc  section  a section  states a consumer reporting company can not furnish an account without my written instructions which i have never given'

In [ ]:
df["Product"].value_counts()

Product
Credit Report    545196
Others           343630
Card             101986
Loan              77295
Services          43223
Name: count, dtype: int64

In [ ]:
df["Product"].value_counts(normalize=True)

Product
Credit Report    0.490580
Others           0.309206
Card             0.091769
Loan             0.069552
Services         0.038893
Name: proportion, dtype: float64

In [ ]:
# Create an empty DataFrame to store sampled data
sampled_df = pd.DataFrame()

# Loop through unique labels in the "Product" column
for label in df["Product"].unique():
    # Filter rows matching the current label
    label_df = df[df["Product"] == label]

    sampled_df = pd.concat([sampled_df,label_df.sample(1000)],ignore_index=True)


In [ ]:
sampled_df

,Product,Consumer complaint narrative
0,Credit Report,i have written citibank twice they say that th...
1,Credit Report,account student loan high credit paid in fu...
2,Credit Report,i have disputed a collection from a total o...
3,Credit Report,company refused to accept payment to close the...
4,Credit Report,so i currently purchased credit repair softwar...
...,...,...
4995,Loan,navient currently i am under an income driven ...
4996,Loan,im in the i got this car for arp which the ...
4997,Loan,i submitted a payment in the amount of on st...
4998,Loan,i attended college from through and got a pri...


In [ ]:
sampled_df["Product"].unique()

array(['Credit Report', 'Others', 'Services', 'Card', 'Loan'],
      dtype=object)

In [ ]:
mapping_dict = {
    'Credit Report': 0,
    'Others': 1,
    'Services': 2,
    'Card': 3,
    'Loan': 4
}


In [ ]:
sampled_df.replace({"Product": mapping_dict},inplace=True)

/tmp/ipykernel_30/698514195.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sampled_df.replace({"Product": mapping_dict},inplace=True)


In [ ]:
sampled_df.columns=['label', 'Complaint']

In [ ]:
sampled_df.head()

,label,Complaint
0,0,i have written citibank twice they say that th...
1,0,account student loan high credit paid in fu...
2,0,i have disputed a collection from a total o...
3,0,company refused to accept payment to close the...
4,0,so i currently purchased credit repair softwar...


In [ ]:
sampled_df.to_csv("preprocessed_data.csv",index=False)

In [ ]:
!pip install datasets

In [ ]:
 from datasets import load_dataset

In [ ]:
from datasets import Dataset

In [ ]:
data=Dataset.from_pandas(sampled_df)

In [ ]:
data

Dataset({
    features: ['label', 'Complaint'],
    num_rows: 5000
})

In [ ]:
train_test=data.train_test_split(test_size=0.2)

In [ ]:
train_test

DatasetDict({
    train: Dataset({
        features: ['label', 'Complaint'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['label', 'Complaint'],
        num_rows: 1000
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_name = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
def tokenize(batch):
  return tokenizer(batch['Complaint'], max_length=500)

In [ ]:
tokenized_data = train_test.map(tokenize, batched=True)    #Loads data in the ram in batches

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'Complaint', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['label', 'Complaint', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(output_dir="output",
                                  eval_strategy="epoch",
                                  num_train_epochs=2)

In [ ]:
def compute_metrics(logits_labels):
  logits, labels = logits_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  return {"accuracy" : acc}

In [ ]:
trainer = Trainer(model,
                  training_args,
                  train_dataset = tokenized_data["train"],
                  eval_dataset=tokenized_data["test"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.704361,0.768000
2,0.754300,0.659916,0.776000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=500, training_loss=0.7543099365234375, metrics={'train_runtime': 510.0734, 'train_samples_per_second': 15.684, 'train_steps_per_second': 0.98, 'total_flos': 1931542061386272.0, 'train_loss': 0.7543099365234375, 'epoch': 2.0})

In [ ]:
model.save_pretrained("finetuned-bert")

In [ ]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained("finetuned-bert", num_labels=5)

In [ ]:
data["label"][0]

0

In [ ]:
data["Complaint"][1]

' account student loan  high credit  paid in full student loan  high credit  currently on time total account  your debttocredit ratio represents the amount of credit youre using and generally makes up a percentage of your credit score its calculated by dividing an account s reported balance by its credit limit \n\nthey report   this is not correct the limit was  the debt to income ratio should be reported at  '

In [ ]:
from transformers import pipeline

In [ ]:
clf=pipeline("text-classification",model=finetuned_model,tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
clf(data["Complaint"][1])